In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import os
import tkinter as tk
from tkinter import filedialog
from mayavi import mlab

In [19]:
#Define the function to request the user to select the folder containing the DICOM files
def select_folder():
    """This function will open a window to select the folder containing the DICOM files"""
    root = tk.Tk()
    root.withdraw()
    folder_selected = filedialog.askdirectory(title='Please select the folder containing the DICOM files',initialdir='C:/')
    return folder_selected

In [20]:
#Define the function to load the DICOM files orderthem by instance number and return the pixel array
def load_dicom(path):
    """This function will load the DICOM files from the selected folder, order them by instance number and return the pixel array of the images with some adjustments in contrast and spacing"""
    #Load the DICOM files
    dicom_files = [pydicom.dcmread(os.path.join(path, file)) for file in os.listdir(path)]
    #Order the DICOM files by instance number
    dicom_files.sort(key=lambda x: int(x.InstanceNumber))
    #Get rescale slope and intercept
    slope = dicom_files[0].RescaleSlope
    intercept = dicom_files[0].RescaleIntercept
    #Return the pixel array
    image = np.stack([file.pixel_array for file in dicom_files])
    image = image * slope + intercept
    return image

In [21]:
#Define the HU range of the tissues of interest
def get_hu_range():
    """This function will return the HU range of the tissues of interest"""
    min_hu = int(input('Enter the minimum HU value: '))
    max_hu = int(input('Enter the maximum HU value: '))
    return min_hu, max_hu

In [22]:
#Define a function to keep only the pixels within the HU range of interest
def keep_hu_range(image, min_hu, max_hu):
    """This function will keep only the pixels within the HU range of interest"""
    image[image < min_hu] = 0
    image[image > max_hu] = 0
    return image

In [23]:
#Define a function to normalize the image
def normalize(image):
    """This function will normalize the image"""
    image -= np.min(image)
    image /= np.max(image)
    return image

In [24]:
#Define a function to plot the 20 slices of the image
def plot_slices(image):
    """This function will plot the 20 slices of the image"""
    plt.clf()
    plt.cla()
    plt.close()
    fig, ax = plt.subplots(4, 5, figsize = (10, 10))
    for i in range(4):
        for j in range(5):
            ax[i, j].imshow(image[5 * i + j], cmap = "gray")
            ax[i, j].axis("off")
    plt.show()
    plt.close()

In [25]:
#Define a function to plot the 3D image using mayavi
def plot_3d(image):
    """This function will plot the 3D image using mayavi to visualize the volume"""
    mlab.figure(bgcolor=(0, 0, 0), size=(800, 800))
    src = mlab.pipeline.scalar_field(image)
    mlab.pipeline.volume(src, vmin=0, vmax=1)
    mlab.show()

In [26]:
#Define a function to create either a 3D contour, a 3D isosurface or a 3D volume selected by the user
def plot_3d_multiple(image):
    """This function will create either a 3D contour, a 3D surface or a 3D volume selected by the user"""
    #Select the type of plot
    plot_type = int(input('Enter the number of the type of plot 1:Contour, 2:Isosurface, 3:Volume: '))
    #Create the plot
    if plot_type == 1:
        mlab.figure(bgcolor=(0, 0, 0), size=(800, 800))
        mlab.contour3d(image, contours=10, transparent=True)
        mlab.show()
    elif plot_type == 2:
        #Create a smooth iso surface
        mlab.figure(bgcolor=(0, 0, 0), size=(800, 800))
        mlab.pipeline.iso_surface(mlab.pipeline.scalar_field(image), contours=[0.5], opacity=0.5,)
        mlab.show()
    elif plot_type == 3:
        mlab.figure(bgcolor=(0, 0, 0), size=(800, 800))
        src = mlab.pipeline.scalar_field(image)
        mlab.pipeline.volume(src, vmin=0, vmax=1)
        mlab.show()
    else:
        print('Invalid option')

In [27]:
#Create a loop to allow the user to use the program multiple times
while True:
    #Request the user to select the folder containing the DICOM files
    path = select_folder()
    #Load the DICOM files
    image = load_dicom(path)
    #Request the user to select the HU range of interest
    min_hu, max_hu = get_hu_range()
    #Keep only the pixels within the HU range of interest
    image = keep_hu_range(image, min_hu, max_hu)
    #Normalize the image
    image = normalize(image)
    #Plot the 20 slices of the image
    #plot_slices(image)
    #Plot the 3D image using mayavi
    plot_3d_multiple(image)
    #Ask the user if he wants to use the program again
    again = input("Do you want to use the program again? (y/n): ")
    if again == "n":
        break

In [28]:
path=select_folder()
image,ds=load_dicom(path)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: ''

In [ ]:
#Read instance number in path
for archivo in os.listdir(path):
    ds=pydicom.dcmread(os.path.join(path,archivo))
    print(ds.InstanceNumber)

In [31]:
#request the user to select one testfile
root = tk.Tk()
root.withdraw()
file_selected = filedialog.askopenfilename(title='Please select the test file',initialdir='C:/')
#Select control file
file_selected2 = filedialog.askopenfilename(title='Please select the control file',initialdir='C:/')

In [32]:
#Read the test file
ds=pydicom.dcmread(file_selected)
#Read the control file
ds2=pydicom.dcmread(file_selected2)

In [35]:
print(ds,ds2)


Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 226
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.9590.100.1.2.365512055907325249631512387333731189448
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.9590.100.1.3.100.9.4
(0002, 0013) Implementation Version Name         SH: 'MATLAB IPT 9.4'
(0002, 0016) Source Application Entity Title     AE: 'bay83ct'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0012) Instance Creation Date              DA: '20200313'
(0008, 0013) Instance Creation Time              TM: '121735'
(0008, 0016) S

In [43]:
#Compare the two metadata and show the differences between them in a table with pandas
#Create a list with the metadata of the test file
metadata_test = [ds.PatientName,ds.PatientID,ds.PatientBirthDate,ds.PatientSex,ds.StudyDate,ds.StudyTime,ds.StudyDescription,ds.SeriesDescription,ds.Modality,ds.Manufacturer,ds.InstitutionName,ds.StationName,ds.SliceThickness,ds.SliceLocation,ds.InstanceNumber,ds.ImagePositionPatient,ds.ImageOrientationPatient,ds.PixelSpacing,ds.WindowCenter,ds.WindowWidth,ds.RescaleIntercept,ds.RescaleSlope,ds.SamplesPerPixel,ds.PhotometricInterpretation,ds.Rows,ds.Columns,ds.BitsAllocated,ds.BitsStored,ds.HighBit,ds.PixelRepresentation,ds.SmallestImagePixelValue,ds.LargestImagePixelValue,ds.PixelData]
#Create a list with the metadata of the control file
metadata_control = [ds2.PatientName,ds2.PatientID,ds2.PatientBirthDate,ds2.PatientSex,ds2.StudyDate,ds2.StudyTime,ds2.StudyDescription,ds2.SeriesDescription,ds2.Modality,ds2.Manufacturer,ds2.InstitutionName,ds2.StationName,ds2.SliceThickness,ds2.SliceLocation,ds2.InstanceNumber,ds2.ImagePositionPatient,ds2.ImageOrientationPatient,ds2.PixelSpacing,ds2.WindowCenter,ds2.WindowWidth,ds2.RescaleIntercept,ds2.RescaleSlope,ds2.SamplesPerPixel,ds2.PhotometricInterpretation,ds2.Rows,ds2.Columns,ds2.BitsAllocated,ds2.BitsStored,ds2.HighBit,ds2.PixelRepresentation,ds2.SmallestImagePixelValue,ds2.LargestImagePixelValue,ds2.PixelData]
#Create a list with the metadata names
metadata_names = ['PatientName','PatientID','PatientBirthDate','PatientSex','StudyDate','StudyTime','StudyDescription','SeriesDescription','Modality','Manufacturer','InstitutionName','StationName','SliceThickness','SliceLocation','InstanceNumber','ImagePositionPatient','ImageOrientationPatient','PixelSpacing','WindowCenter','WindowWidth','RescaleIntercept','RescaleSlope','SamplesPerPixel','PhotometricInterpretation','Rows','Columns','BitsAllocated','BitsStored','HighBit','PixelRepresentation','SmallestImagePixelValue','LargestImagePixelValue','PixelData']
#Create a dataframe with the metadata of the test file
df_test = pd.DataFrame(metadata_test, index=metadata_names, columns=['Test'])
#Create a dataframe with the metadata of the control file
df_control = pd.DataFrame(metadata_control, index=metadata_names, columns=['Control'])
#Create a dataframe with the differences between the metadata of the test file and the control file
df_diff = pd.concat([df_test, df_control], axis=1, sort=False)
df_diff['Difference'] = df_diff['Test'] != df_diff['Control']
#Show the differences between the metadata of the test file and the control file
df_diff

AttributeError: 'FileDataset' object has no attribute 'PatientName'